In [3]:
import pandas as pd
import re

In [5]:
# Upload the .tsv
FILEPATH = 'train.tsv'
training_df = pd.read_csv(FILEPATH, sep='\t').dropna()

training_df.sample(10)
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 323
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   310 non-null    object
 1   JLMelenchon  310 non-null    object
dtypes: object(2)
memory usage: 7.3+ KB


In [29]:
files = ['CBeaune', 'CZacharopoulou', 'franckriester', 'JLMelenchon', 'MinColonna']

# For cleaning purposes
dataset = {file: '' for file in files}

for file in files:
    with open(file + '.txt', 'r', encoding='utf-8') as src:
        dataset[file] = src.read(-1)
        print("SUBJECT:", file)
        print(dataset[file][:100])

SUBJECT: CBeaune
ts | En cette journée mondiale du #DonDeSang  , rappelons que la majorité a mis fin à une inégalité 
SUBJECT: CZacharopoulou
                           ·  21h            Excellente discussion avec S
E   @IssoufouMhm  , ancien
SUBJECT: franckriester
                  @      ·  Jun 9            « Il y a besoin d’un pays attractif, où les investissem
SUBJECT: JLMelenchon
  Retweeted           L'insoumission      @L_insoumission    ·  1h  « Qu'avez-vous ressenti devant c
SUBJECT: MinColonna
                   @     ·  Jun 10              En #Touraine, avec les candidats de la majorité
 Mer


In [30]:
# Cleaning data and making dataframe
dataset_df = pd.DataFrame(columns=['tweet', 'author'])

for author in dataset.keys():
    dataset[author] = re.sub(r'\s{2,}|\|', ' ', dataset[author]).split('views')
    for tweet in dataset[author]:
        dataset_df = dataset_df.append({'tweet': tweet, 'author': author}, ignore_index=True)

dataset_df.sample(10) 

,tweet,author
38,0:00 / 1:11 33 201 587 @ · Jun 10 « Ce dimanc...,JLMelenchon
5,0:01 / 1:56 44 114 246 @ · Jun 13 Juste une m...,CBeaune
11,0:00 / 0:56 305 364 692 Retweeted Olivia Greg...,CZacharopoulou
3,0:01 / 1:02 47 56 122 @ · 11h Une majorité po...,CBeaune
28,0:01 / 1:22 6 21 Retweeted avec vous @avecvou...,franckriester
18,0:00 / 1:22 17 36 101 Retweeted Emmanuel Macr...,CZacharopoulou
23,0:01 / 4:50 31 172 421 Retweeted Emmanuel Mac...,CZacharopoulou
39,"0:45 / 2:23 148 443 1,052 Retweeted Adrien Qu...",JLMelenchon
24,"343 725 2,275 Retweeted Emmanuel Macron @Emma...",CZacharopoulou
42,"0:01 / 1:55 44 468 1,097 Retweeted Nouvelle U...",JLMelenchon


In [ ]:
#Save it as a pickle
dataset_df.to_pickle('dataset_df.pkl')